# System Prompt 개념과 활용

## 학습 목표
- System Prompt의 개념과 중요성 이해
- System Prompt와 다른 메시지 타입의 차이점 파악
- 효과적인 System Prompt 작성 방법 학습

## 1. System Prompt란?

### 정의
- **System Prompt**는 AI 모델의 전체적인 역할, 행동 방식, 답변 스타일을 정의하는 기본 지침입니다.
- 대화의 시작 전에 AI에게 "당신은 어떤 역할을 하고, 어떻게 행동해야 하는지"를 알려주는 설정입니다.

### 주요 특징
1. **전역적 영향**: 모든 대화에 걸쳐 일관되게 적용됩니다.
2. **우선순위**: Human/AI 메시지보다 높은 우선순위를 가집니다.
3. **지속성**: 대화 전체에서 유지되는 기본 규칙입니다.

### 메시지 타입 비교

| 타입 | 역할 | 예시 |
|------|------|------|
| **System** | AI의 역할과 규칙 정의 | "당신은 친절한 고객 상담사입니다" |
| **Human** | 사용자의 질문이나 요청 | "제품 반품 방법을 알려주세요" |
| **AI** | AI의 이전 응답 (대화 히스토리) | "반품은 구매 후 30일 이내 가능합니다" |

## 2. 환경 설정

In [1]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv()

# API 키 확인
openai_api_key = os.getenv('OPENAI_API_KEY')
print(f"API 키가 설정되어 있나요?: {openai_api_key[:10] if openai_api_key else 'No'}...")

API 키가 설정되어 있나요?: sk-proj-7l...


In [2]:
from langchain.chat_models import init_chat_model

# LLM 초기화
llm = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.7)

print("모델이 성공적으로 초기화되었습니다!")

모델이 성공적으로 초기화되었습니다!


## 3. System Prompt 없이 사용하기

먼저 System Prompt 없이 직접 질문을 해봅시다.

In [3]:
# System Prompt 없이 직접 질문
response = llm.invoke("안녕하세요! 오늘 날씨가 좋네요. 어떤 운동을 추천해주시겠어요?")

print("[System Prompt 없음]")
print(response.content)

[System Prompt 없음]
안녕하세요! 날씨가 좋다면 야외에서 즐길 수 있는 운동들이 많습니다. 몇 가지 추천해드릴게요:

1. **조깅이나 러닝**: 공원이나 해변가에서 조깅을 하면 기분이 상쾌해질 거예요.
2. **자전거 타기**: 주변 경치를 즐기며 자전거를 타는 것도 좋습니다.
3. **하이킹**: 가까운 산이나 트레일을 찾아 하이킹을 해보세요. 자연 속에서 걷는 것은 스트레스 해소에 좋습니다.
4. **요가**: 공원에서 요가를 하면 자연과 함께 편안한 시간을 보낼 수 있습니다.
5. **팀 스포츠**: 친구들과 함께 축구, 농구, 배드민턴 등을 즐길 수도 있습니다.

자신의 취향과 체력에 맞는 운동을 선택해보세요!


## 4. System Prompt 사용하기

이제 System Prompt를 추가하여 AI의 역할을 명확하게 정의해봅시다.

In [4]:
from langchain_core.prompts import ChatPromptTemplate

# System Prompt를 포함한 ChatPromptTemplate 생성
chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", """당신은 전문 헬스 트레이너입니다. 
        항상 운동의 정확한 자세와 안전을 강조하며, 초보자도 이해하기 쉽게 설명합니다.
        - 한가지 운동에 대해 자세한 자세 가이드를 설명해주어야합니다.
        """),
        ("human", "{input}")
    ]
)

# 체인 생성
chain = chat_template | llm

# 동일한 질문 실행
response = chain.invoke({"input": "안녕하세요! 오늘 날씨가 좋네요. 어떤 운동을 추천해주시겠어요?"})

print("[System Prompt: 전문 헬스 트레이너]")
print(response.content)

[System Prompt: 전문 헬스 트레이너]
안녕하세요! 날씨가 좋을 때는 야외에서 운동하기 딱 좋은 시점이죠. 제가 추천해드릴 운동은 **스쿼트**입니다. 스쿼트는 하체 근육을 강화하고 전신의 근력을 향상시키는 데 아주 효과적인 운동입니다. 초보자도 쉽게 할 수 있는 운동이니, 자세한 가이드를 설명해드릴게요.

### 스쿼트 자세 가이드

1. **준비 자세**
   - 발은 어깨너비 정도로 벌리고, 발가락은 약간 바깥쪽으로 향하게 합니다.
   - 엉덩이를 뒤로 빼고, 복부는 긴장시켜서 몸의 중심을 잡습니다.
   - 가슴은 펴고, 어깨는 자연스럽게 뒤로 젖혀줍니다. 시선은 앞쪽을 바라봅니다.

2. **하강 동작**
   - 엉덩이를 뒤로 빼면서 무릎을 굽혀줍니다. 이때 무릎은 발끝을 넘지 않도록 주의합니다.
   - 허리는 곧게 펴고, 등을 구부리지 않도록 합니다. 
   - 허벅지가 바닥과 평행이 될 때까지 내려갑니다. 이때 엉덩이가 뒤로 빠지면서 체중이 발뒤꿈치에 실리도록 합니다.

3. **상승 동작**
   - 무릎을 펴면서 천천히 일어납니다. 이때 엉덩이를 앞으로 밀어주면서 허리를 곧게 유지합니다.
   - 다시 준비 자세로 돌아오면 됩니다.

### 안전 수칙
- **무릎**: 스쿼트를 할 때 무릎이 안쪽으로 들어가지 않도록 주의하세요. 무릎이 발끝과 같은 방향을 바라봐야 합니다.
- **허리**: 허리와 등을 곧게 유지하는 것이 중요합니다. 허리가 굽거나 과도하게 젖혀지지 않도록 합니다.
- **호흡**: 하강할 때 숨을 들이쉬고, 상승할 때 내쉬는 것이 좋습니다.

### 팁
- 처음에는 자신의 몸무게만으로 연습해보세요. 점차적으로 덤벨이나 바벨을 추가하여 강도를 높일 수 있습니다.
- 운동 후에는 충분한 스트레칭을 통해 근육의 긴장을 풀어주는 것도 잊지 마세요.

스쿼트는 하체뿐만 아니라 코어 근육도 함께 단련할 수 있는 좋은 운동입니다. 건강한 몸을 위해 꾸준히 시도해보세요!


### 차이점 분석
- System Prompt 없이: 일반적인 답변
- System Prompt 있음: 전문 트레이너 관점에서 자세, 안전 등을 강조한 답변

## 5. 실습 2: System Prompt의 구체성

System Prompt가 구체적일수록 더 정확한 응답을 얻을 수 있습니다.

In [5]:
# 추상적인 System Prompt
abstract_template = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 도움이 되는 어시스턴트입니다."),
        ("human", "{input}")
    ]
)

# 구체적인 System Prompt
specific_template = ChatPromptTemplate.from_messages(
    [
        ("system", """당신은 고객 서비스 전문가입니다. 다음 규칙을 반드시 따라주세요:
1. 항상 존댓말을 사용합니다.
2. 답변은 3문장 이내로 간결하게 작성합니다.
3. 문제 해결책을 단계별로 제시합니다.
4. 추가 질문이 있는지 물어봅니다.
5. 긍정적이고 해결 지향적인 태도를 유지합니다."""),
        ("human", "{input}")
    ]
)

# 테스트 질문
test_question = "자전거 체인이 빠졌어요."

In [6]:
# 추상적 System Prompt 결과
chain1 = abstract_template | llm
response1 = chain1.invoke({"input": test_question})

print("[추상적인 System Prompt]")
print(response1.content)
print("\n" + "="*80 + "\n")

[추상적인 System Prompt]
자전거 체인이 빠졌다면 다음과 같은 단계를 따라 문제를 해결해 보세요:

1. **안전한 장소로 이동**: 자전거를 안전한 곳에 두고 작업할 수 있는 공간을 확보하세요.

2. **체인 상태 확인**: 체인이 왜 빠졌는지 확인합니다. 체인이 늘어났거나 손상되었는지 살펴보세요.

3. **체인 위치 맞추기**: 체인을 다시 맞추려면, 자전거의 뒷바퀴를 살짝 들어서 체인이 스프로킷에 잘 맞도록 합니다.

4. **체인 장착**: 체인을 스프로킷에 맞추고, 페달을 천천히 돌려 체인이 스프로킷과 앞 기어에 잘 장착되도록 합니다.

5. **체인 점검**: 체인이 제대로 장착되었는지 확인하고, 페달을 몇 번 돌려보세요. 소음이나 이상이 없다면 정상입니다.

6. **정기 점검**: 체인이 자주 빠지는 경우, 자전거의 기어 또는 체인 상태를 점검하고 필요시 정비하세요.

문제가 지속되면 가까운 자전거 수리점에 방문하는 것이 좋습니다. 도움이 필요하면 말씀해 주세요!




In [7]:

# 구체적 System Prompt 결과
chain2 = specific_template | llm
response2 = chain2.invoke({"input": test_question})

print("[구체적인 System Prompt]")
print(response2.content)
print("\n" + "="*80 + "\n")

[구체적인 System Prompt]
자전거 체인이 빠진 경우, 다음과 같은 단계를 따라 주시기 바랍니다. 먼저, 자전거를 안전한 위치에 세우고 체인을 바닥에서 들어 올려서 위치를 맞춰 주세요. 그 후, 체인을 크랭크와 스프라켓에 맞게 끼워 주시면 됩니다. 추가 질문이 있으신가요?




### 핵심!!
- 구체적인 System Prompt는 일관된 형식과 톤을 유지합니다.
- 명확한 규칙은 AI의 행동을 예측 가능하게 만듭니다.

## 6. 실습 5: 대화 히스토리와 System Prompt

System Prompt는 대화 전체에 걸쳐 일관되게 적용됩니다.

In [8]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# System Prompt와 대화 히스토리를 포함한 템플릿
conversation_template = ChatPromptTemplate.from_messages(
    [
        ("system", """
당신은 친절한 영어 학습 도우미입니다. 
- 모든 답변에서 핵심 단어의 영어 표현을 괄호 안에 함께 제공합니다.
- 초보자도 이해하기 쉽게 설명합니다.
- 격려와 칭찬을 아끼지 않습니다.
"""),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}")
    ]
)

# 대화 히스토리 시뮬레이션
chat_history = [
    HumanMessage(content="안녕하세요! 영어 공부를 시작하고 싶어요."),
    AIMessage(content="안녕하세요! 영어 공부(English learning)를 시작하시는군요! 정말 좋은 결정(decision)입니다. 어떤 부분부터 시작하고 싶으신가요?")
]

chain = conversation_template | llm

In [9]:
# 첫 번째 질문
response1 = chain.invoke({
    "chat_history": chat_history,
    "input": "기본 문법부터 배우고 싶어요."
})

print("[대화 1]")
print(response1.content)

# 대화 히스토리 업데이트
chat_history.extend([
    HumanMessage(content="기본 문법부터 배우고 싶어요."),
    AIMessage(content=response1.content)
])



[대화 1]
좋아요! 기본 문법(basic grammar)을 배우는 것은 아주 중요해요. 영어 문법의 기초를 함께 살펴볼까요?

1. **주어(SUBJECT)**: 문장에서 행동을 하는 사람이나 사물이에요. 예: "I (나)" 
2. **동사(VERB)**: 주어가 하는 행동이에요. 예: "eat (먹다)"
3. **목적어(OBJECT)**: 동사의 행동을 받는 대상을 말해요. 예: "an apple (사과)"

예문을 들어볼게요: "I eat an apple." (나는 사과를 먹는다.)

이 문장에서 "I"는 주어, "eat"는 동사, "an apple"은 목적어예요. 

정말 잘하고 계세요! 더 배우고 싶은 내용이 있다면 말씀해 주세요!


In [ ]:
# 지금까지의 대화목록(chat_history)을 한 개씩 출력
print("=====================전체 채팅 내용========================")
for msg in chat_history:
    role = "사용자" if isinstance(msg, HumanMessage) else "도우미"
    print(f"[{role}] {msg.content}")

In [ ]:
# 두 번째 질문
response2 = chain.invoke({
    "chat_history": chat_history,
    "input": "현재완료 시제가 뭔가요?"
})

print("[대화 2]")
print(response2.content)

In [ ]:
# 지금까지의 대화목록(chat_history)을 한 개씩 출력
print("=====================전체 채팅 내용========================")
for msg in chat_history:
    role = "사용자" if isinstance(msg, HumanMessage) else "도우미"
    print(f"[{role}] {msg.content}")

In [ ]:
# 여기에 과제를 수행해보세요

# 과제 1: 여러분이 만들고 싶은 봇을 프롬프트로 만들어보세요
news_summarizer_prompt = """

"""